In [21]:
from __future__ import division
import pandas as pd
import numpy as np 
import json, os, sys
import statsmodels.api as sm
from scipy.stats import poisson

In [42]:
raw_df = pd.read_clipboard()

In [49]:
df = raw_df
df['team'] = df['team'].astype('category')
df['opponent_team'] = df['opponent_team'].astype('category')


print df.info()

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 6 columns):
date                    760 non-null object
time                    760 non-null object
team                    760 non-null category
opponent_team           760 non-null category
goals_scored_by_team    760 non-null int64
home_flag               760 non-null int64
dtypes: category(2), int64(2), object(2)
memory usage: 25.6+ KB
None


,date,time,team,opponent_team,goals_scored_by_team,home_flag
0,17/5/2016,20:00,Man Utd,Bournemouth,3,0
1,15/5/2016,15:00,Arsenal,Aston Villa,4,0
2,16/5/2016,15:00,Chelsea,Leicester,1,0
3,17/5/2016,15:00,Everton,Norwich,3,0
4,18/5/2016,15:00,Newcastle,Spurs,5,0




## $ Pr (Home Goals) \sim Poisson(\lambda) $

## $ Pr(Away Goals) \sim Poisson (\mu) $



## $ \lambda \sim \gamma \tau \alpha_{i} \beta_{j} $

## $ \mu \sim \gamma \alpha_{j} \beta_{i} $

##### We can initialize $\gamma $ as half the mean of total goals divided by 2. (To give avg number of goals per team, as each game contains 2 teams

In [20]:
gamma = np.mean(df['home_score'] + df['away_score']) / 2

##### We can estimate the params for $ \alpha$ and $ \beta $ using GLM from statsmodels

In [147]:
m = smf.glm(formula='goals_scored_by_team ~ home_flag + team + opponent_team', data=df, family=sm.families.Poisson())
res = m.fit()

res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                  Generalized Linear Model Regression Results                   
================================================================================
Dep. Variable:     goals_scored_by_team   No. Observations:                  760
Model:                              GLM   Df Residuals:                      720
Model Family:                   Poisson   Df Model:                           39
Link Function:                      log   Scale:                             1.0
Method:                            IRLS   Log-Likelihood:                -1114.7
Date:                  Sat, 16 Jul 2016   Deviance:                       893.75
Time:                          23:33:53   Pearson chi2:                     762.
No. Iterations:                       8                                         
===================================================================================================
                                      coef    std err          z      P>|z|      [95.0% Conf. Int.]
---------------------------------------------------------------------------------------------------
Intercept                           0.3384      0.206      1.639      0.101        -0.066     0.743
team[T.Aston Villa]                 0.1494      0.211      0.710      0.478        -0.263     0.562
team[T.Bournemouth]                 0.3022      0.204      1.484      0.138        -0.097     0.701
team[T.Chelsea]                     0.3813      0.200      1.905      0.057        -0.011     0.774
team[T.Crystal Palace]             -0.0112      0.218     -0.051      0.959        -0.439     0.417
team[T.Everton]                     0.4277      0.198      2.157      0.031         0.039     0.816
team[T.Leicester]                   0.2250      0.207      1.088      0.277        -0.180     0.630
team[T.Liverpool]                   0.2694      0.205      1.313      0.189        -0.133     0.672
team[T.Man City]                    0.4678      0.197      2.380      0.017         0.083     0.853
team[T.Man Utd]                    -0.1657      0.227     -0.729      0.466        -0.611     0.280
team[T.Newcastle]                   0.2823      0.204      1.381      0.167        -0.118     0.683
team[T.Norwich]                     0.2792      0.204      1.366      0.172        -0.121     0.680
team[T.Southampton]                 0.3537      0.201      1.762      0.078        -0.040     0.747
team[T.Spurs]                       0.1697      0.210      0.810      0.418        -0.241     0.580
team[T.Stoke]                       0.0820      0.214      0.384      0.701        -0.337     0.501
team[T.Sunderland]                  0.0319      0.217      0.147      0.883        -0.394     0.458
team[T.Swansea]                    -0.0540      0.221     -0.244      0.807        -0.488     0.380
team[T.Watford]                    -0.0824      0.223     -0.370      0.711        -0.519     0.354
team[T.West Brom]                   0.0678      0.214      0.317      0.751        -0.351     0.487
team[T.West Ham]                    0.3547      0.201      1.760      0.078        -0.040     0.750
opponent_team[T.Aston Villa]       -0.0816      0.189     -0.433      0.665        -0.451     0.288
opponent_team[T.Bournemouth]       -0.0549      0.188     -0.293      0.770        -0.423     0.313
opponent_team[T.Chelsea]           -0.1453      0.193     -0.755      0.450        -0.523     0.232
opponent_team[T.Crystal Palace]    -0.2068      0.195     -1.063      0.288        -0.588     0.175
opponent_team[T.Everton]           -0.1625      0.194     -0.839      0.401        -0.542     0.217
opponent_team[T.Leicester]         -0.1348      0.191     -0.704      0.481        -0.510     0.240
opponent_team[T.Liverpool]         -0.0037      0.185     -0.020      0.984        -0.367     0.359
opponent_team[T.Man City]          -0.2674      0.199     -1.340      0.180        -0.658     0.124
opponent_team[T.Man Utd]           -0.2129

In [89]:
home_team = 'Aston Villa'
away_team = 'Bournemouth'

np.exp(m.predict(-0.263269, 0.701311)) - np.exp(m.predict(0.562123, -0.096996))

-0.28124711488600163

In [141]:
test_list = []
for i in xrange(1000):
    test_list.append(m.predict(-0.263269, 0.701311, 0))

In [116]:
coef_df.loc['team[T.Bournemouth]']

0   -0.096996
1    0.701311
Name: team[T.Bournemouth], dtype: float64

In [90]:
def get_expected_goals(home_team, away_team, coef_df, model):
    home_attack_score = np.exp(coef_df.loc['team[T.{0}]'.format(home_team)][0])
    home_defence_score = np.exp(coef_df.loc['team[T.{0}]'.format(home_team)][1])
    
    away_attack_score = np.exp(coef_df.loc['team[T.{0}]'.format(away_team)][0])
    away_defence_score = np.exp(coef_df.loc['team[T.{0}]'.format(home_team)][1])
    
    attack_outcome = model.predict()
    
    poisson()

In [112]:
np.exp(-0.28124711488600163)

0.75484177979045119

In [126]:
m.predict(0.701311).shape

(760, 40)

In [123]:
poisson(np.exp(m.predict(-0.263269, 0.701311))).rvs(1)

array([1])

In [ ]:
poisson(np.exp(m.predict(0.701311)))